#  Programowanie dla analityki danych -- Projekt

## 1 -- Wybor tematu

Temat (WIP)

## 2 -- Zebranie/znalezienie danych (Web scrapping)

In [19]:
from selenium import webdriver
import time
import csv

def get_data_from_website():
    leagues = []


    # -- Zaladowanie webdrivera i polaczenie sie z strona --
    driver = webdriver.Chrome()
    driver.get('https://poe-antiquary.xyz/Archnemesis/Currency/Exalted%20Orb/2')


    # -- Usuniecie reklamy
    all_ins = driver.find_elements_by_tag_name("ins")
    if len(all_ins) > 0: driver.execute_script("""var elems = document.getElementsByTagName("ins"); 
                            for(var i = 0, max = elems.length; i < max; i++) {elems[i].hidden=true;}""")


    # -- Wypisanie wszystkich lig na grafie --
    driver.find_element_by_xpath('//button[@data-id="league-select"]').click()
    for league in driver.find_elements_by_xpath('//div[@class="dropdown-menu show"]/div[@class="inner show"]/ul/li/a'):
        if(league.text != "Archnemesis"):
            league.click()
            driver.find_element_by_xpath('//label[@class="btn btn-primary form-control"]').click()
            driver.find_element_by_xpath('//button[@data-id="league-select"]').click()
    time.sleep(2)


    # -- Wrzucenie wszystkich informacji o ligach do tablicy --
    for line in driver.find_elements_by_xpath('//*[name()="svg"]/*[name()="g"]/*[name()="g"]/*[name()="line"]'):
        leagues.append([line.get_attribute("ct:value"),line.get_attribute("ct:meta")])


    # -- Zapisanie tablicy do pliku CSV
    with open('Leagues.csv', 'w', newline='', encoding='utf-8') as f:
            write = csv.writer(f, delimiter=';')
            write.writerow(['Time(h)/Currency(c)','Description'])
            write.writerows(leagues)

get_data_from_website()


C:\Users\Mati\AppData\Local\Temp\ipykernel_20656\2887035650.py:15: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  all_ins = driver.find_elements_by_tag_name("ins")
C:\Users\Mati\AppData\Local\Temp\ipykernel_20656\2887035650.py:21: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//button[@data-id="league-select"]').click()
C:\Users\Mati\AppData\Local\Temp\ipykernel_20656\2887035650.py:22: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  for league in driver.find_elements_by_xpath('//div[@class="dropdown-menu show"]/div[@class="inner show"]/ul/li/a'):
C:\Users\Mati\AppData\Local\Temp\ipykernel_20656\2887035650.py:25: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_el

## 3 -- Czyszczenie danych/dodanie brakujących wartości

In [121]:
import pandas as pd

def data_cleaning(df):
    new_df = df

    # -- Rozdzielenie kolumny "Time/Currency" na 2 osobne kolumny "Hour" i "Price" -- 
    new_df[['Hour','Price']] = new_df['Time(h)/Currency(c)'].str.split(',', expand=True)
    new_df[['Hour','Price']] = new_df[['Hour','Price']].apply(pd.to_numeric)
    new_df = new_df.drop(columns = ['Time(h)/Currency(c)'])


    # -- Rozdzielenie kolumny "Description" na 2 osobne kolumny "League" i "Day" --
    new_df['Day'] = new_df['Description'].str.extract(r'([0-9]+)')
    new_df['Day'] = new_df['Day'].apply(pd.to_numeric)
    new_df['League'] = new_df['Description'].str.extract(r'(in [a-zA-Z]+.*[a-zA-Z]+:)')
    new_df['League'] =  new_df['League'].str[3:-1]
    new_df = new_df.drop(columns = ['Description'])

    
    # -- Rozdzielenie kolumny "League" na 2 osobne kolumny "League Type" i "League Lenght" --
    new_df['League Type'] = new_df.apply(lambda x: 'Hardcore' if 'HC' in str(x) else 'Standard', axis = 1)
    new_df['League'] =  new_df['League'].apply(lambda x: str(x).replace(' HC','') if 'HC' in str(x) else x)
    new_df['League Length'] = new_df.apply(lambda x: 'Short' if '(' in str(x) else 'Long', axis = 1)


    return new_df

In [104]:
import pandas as pd

# -- Zaladowanie pliku CSV do dataframe'a --

df_before = pd.read_csv("Leagues.csv",sep=";")
df_before

,Time(h)/Currency(c),Description
0,"3,33.73",Price after 0 day in Archnemesis:
1,"27,56.37",Price after 1 day in Archnemesis:
2,"51,65.53",Price after 2 days in Archnemesis:
3,"75,73.5",Price after 3 days in Archnemesis:
4,"99,77.8",Price after 4 days in Archnemesis:
...,...,...
12089,"2187,109.48",Price after 91 days in Archnemesis HC:
12090,"2211,110.66",Price after 92 days in Archnemesis HC:
12091,"2235,111.54",Price after 93 days in Archnemesis HC:
12092,"2259,112.34",Price after 94 days in Archnemesis HC:


In [122]:
df_after = data_cleaning(df_before)
df_after
    

,Hour,Price,Day,League,League Type,League Length
0,3,33.73,0,Archnemesis,Standard,Long
1,27,56.37,1,Archnemesis,Standard,Long
2,51,65.53,2,Archnemesis,Standard,Long
3,75,73.50,3,Archnemesis,Standard,Long
4,99,77.80,4,Archnemesis,Standard,Long
...,...,...,...,...,...,...
12089,2187,109.48,91,Archnemesis,Hardcore,Long
12090,2211,110.66,92,Archnemesis,Hardcore,Long
12091,2235,111.54,93,Archnemesis,Hardcore,Long
12092,2259,112.34,94,Archnemesis,Hardcore,Long


## 4 -- Analiza danych

## 5 -- Stworzenie dashboardu
Dashboard utworzony za pomoca streamlit'a.

##  6 -- Utworzenie modelu i ewaluacja
Model regresyjny